# Data transformation

Description: data transformation and missing value handling

In [ ]:
# set project and working directory
home_dir = os.environ['project_home']

### === MAIN ===

#### ==== parameters setting ====

In [ ]:
import datetime
cur_date = datetime.datetime.now()
parameter_01 = 0
parameter_02 = 0
parameter_03 = 0

### Read data

In [ ]:
data_source_dir = working_dir+'/vars/'

# select latest dated file (track record)
cmd = 'ls {data_source_dir}/Sessions.*.pickle'.format(data_source_dir=data_source_dir)
input_filepath = commands.getoutput(cmd).splitlines()[-1]
__nb_logger.info('reading file: %s' % input_filepath)
Sessions = pickle.load(open(input_filepath))

#### Filling missing values

In [ ]:
fillers = {
  'contextual_data': {
    'feature_01': 'default_01',
    'feature_02': 'default_02',
    'feature_03': 'default_03',
  },
  'track_records': {
    'feature_04': 'default_04',
    'feature_05': 'default_05',
    'feature_06': 'default_06',        
  }
} # end filler

In [ ]:
for _id in Sessions.keys():
    # contextual
    for key, val in fillers['contextual_data'].iteritems():
        Sessions[_id]['contextural_data'][key].fillna(val, inplace=True)
    # end for
    
    # track-records
    for key, val in fillers['track_records'].iteritems():
        Sessions[_id]   ['track_records'][key].fillna(val, inplace=True)
    # end for
# end for

#### Dates

In [ ]:
import dateutil
colnames = ['date_feature_01', 'date_feature_02', 'date_feature_03']

In [ ]:
for _id in Sessions.keys():
    for colname in colnames:
        dt_vals = Sessions[_id]['track_records'][colname].values
        ds = cur_date - np.array(map(dateutil.parser.parse, dt_vals))
        Sessions[_id]['track_records']['days_past_'+colname] = ds
    # end for
# end for

#### Log-transformation

In [ ]:
colnames = ['feature_01', 'feature_02', 'feature_03']

In [ ]:
for _id in Sessions.keys():
    for colname in colnames:
        dt_vals = Sessions[_id]['track_records'][colname].values
        ds = np.log(1 + dt_vals)
        Sessions[_id]['track_records']['log_'+colname] = ds
    # end for
# end for

### Export to file

In [ ]:
# specify output
outfilename = working_dir + '/vars/Sessions.%s.pickle' % int(utc_timestamp())
__nb_logger.info('write to output: %s' % outfilename)

In [ ]:
pickle.dump(Sessions, open(outfilename, 'w'))